# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [31]:
#Your code here
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import numpy as np
import statsmodels as sum
from scipy import stats


In [2]:
#import dataset
df =pd.read_csv('homepage_actions.csv')
df.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


looking at the data

In [3]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [4]:
exp = df[df.group == 'experiment']
# contrl = df[df.group == 'control']

exp
# contrl

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
...,...,...,...,...
8179,2017-01-18 08:53:50.910310,615849,experiment,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view


In [5]:
contrl = df[df.group == 'control']
contrl

,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view
...,...,...,...,...
8178,2017-01-18 08:17:12.675797,616692,control,view
8181,2017-01-18 09:07:37.661143,795585,control,view
8182,2017-01-18 09:09:17.363917,795585,control,click
8186,2017-01-18 10:08:51.588469,505451,control,view


In [6]:
exp.id.value_counts()


id
323922    2
404540    2
920069    2
609072    2
252138    2
         ..
290229    1
422224    1
866782    1
863941    1
458115    1
Name: count, Length: 2996, dtype: int64

In [7]:
view = df[df.action == 'view']

view

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [8]:
click = df[df.action == 'click']
click

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8180,2017-01-18 08:54:56.879682,615849,experiment,click


In [9]:
# view = df[df.action == 'view']
# click = df[df.action == 'click']

In [10]:
#checking if anyone clicked but didn't view
click['id'].isin(view['id']).sum() == len(click.id)


True

In [11]:
# comments abt the observation, if a person clicked and din't view
"we can see that, everyone that clicked also viewed"

'we can see that, everyone that clicked also viewed'

In [12]:
#checking for overlap between the two groups ie 
#if there are ids in the experimental group that are in the control group

np.isin(exp.id.unique(),contrl.id.unique()).sum()

0

In [13]:
#checking for overlap between the two groups ie 
#if there are ids in the control group that are in the experiment group

np.isin(contrl.id.unique(),exp.id.unique()).sum()

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [18]:
#Your code here
print(exp.shape,contrl.shape)

(3924, 4) (4264, 4)


In [17]:
"""
From the two groups of sample sizes
h_0: There is no difference between experimental and control homepages
h_1: The experimental homepage is more effective than the control homepage
"""

'\nFrom the two groups of sample sizes\nh_0: There is no difference between experimental and control homepages\nh_1: The experimental homepage is more effective than the control homepage\n'

In [26]:
exp ['click'] = 0
exp.loc [exp.action =='click', 'click'] =1


C:\Users\user\AppData\Local\Temp\ipykernel_8212\3231952382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp ['click'] = 0


In [22]:
contrl ['click'] = 0
contrl.loc [contrl.action == 'click','click'] = 1

C:\Users\user\AppData\Local\Temp\ipykernel_8212\1215188584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contrl ['click'] = 0


In [28]:
contrl.click.value_counts()


click
0    2996
1     928
Name: count, dtype: int64

In [32]:
stats.ttest_ind(contrl.click, exp.click, equal_var=False, alternative='less')

Ttest_indResult(statistic=-1.9312441445072492, pvalue=0.026743886922199395)

In [33]:
"""
The p value is smaller than the alpha therefore we can reject the null hypothesis
"""

'\nThe p value is smaller than the alpha therefore we can reject the null hypothesis\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [39]:
#Your code here
no_of_clicks = contrl.click.sum()/len(contrl.click)
exp_clicks= no_of_clicks * len(exp.click)
exp_clicks

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [36]:
#Your code here
no_of_clicks

0.21857410881801126

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [37]:
#Your code here

#Your code here
n = len(exp.click)
p = no_of_clicks
v = n*p*(1-p)
std = np.sqrt(v)

In [40]:
z=(exp.click.sum()-exp_clicks)/std

In [41]:
#Your code here
1-stats.norm.cdf(z)

0.0033030672759265522

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p-value in both tests is lower than the alpha of 0.05 and hence we can reject the null hypothesis.
 The experimental homepage is indeed more effective than the control homepage.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.